In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
from matplotlib import cm

def make_map(projection=ccrs.PlateCarree()):
    
    fig, ax = plt.subplots(figsize=(16, 6),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
cmap_wl = cm.Blues
N = matplotlib.colors.Normalize(0,2)

In [ ]:
# test Glofrim BMI
from glofrim import DFM
config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/Elbe_1way_1d2dFM_400m_3200m.mdu'
engine = r'/home/dirk/Models/dflowfm-1.1.201/lib/libdflowfm.so'
bmi = DFM(engine = engine)
bmi.initialize(config_fn=config_fn)
bmi = bmi._bmi
# fcoords = bmi.get_grid().get_poly_coords()

In [ ]:
# import netCDF4
# import pyugrid

# print('pyugrid version: {}'.format(pyugrid.__version__))
# print('netCDF4 version: {}'.format(netCDF4.__version__))

# fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/1d2dFM_Elbe_fine3_emb_net.nc'
# fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/test_net.nc'
# ug = pyugrid.UGrid.from_ncfile(fn)
# ug.nodes
# ug.faces

In [ ]:
# from bmi.wrapper import BMIWrapper
# engine = r'/home/dirk/Models/dflowfm-1.1.201/lib/libdflowfm.so'
# config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/Elbe_1way_1d2dFM_400m_3200m.mdu'
# bmi = BMIWrapper(engine = engine)
# bmi.initialize(config_fn)

In [ ]:
[bmi.get_var_name(i) for i in range(bmi.get_var_count()) if '' in bmi.get_var_name(i)]

https://repos.deltares.nl/repos/ds/trunk/additional/unstruc/src/bmi_get_var.inc

#### questions
- what is the difference between xz,yz and xk,yk?
- 

In [ ]:
# all cells 2d + 1d

cell_x = bmi.get_var('xz') # x-coords of each cell centre point
cell_y = bmi.get_var('yz') # y-coords of each cell centre point
cell_xy = np.array(zip(cell_x, cell_y))
ndx2d = int(bmi.get_var('ndx2d'))
ndxi = int(bmi.get_var('ndxi'))
cidx_2d = np.arange(ndx2d)
cidx_1d = np.arange(ndx2d, ndxi)
cidx_bnds = np.arange(ndxi, cell_xy.shape[0])

face_coordinates = cell_xy[cidx_2d, :] # coordinates of cell centres

In [ ]:
# 2d mesh 
node_lon = bmi.get_var('xk')
node_lat = bmi.get_var('yk') 
nodes = np.array(zip(node_lon, node_lat))

faces = bmi.get_var('flowelemnode') # face_node_connectivity
faces = np.ma.masked_equal(faces, -1) - 1

nidx_2d = np.arange(faces.max()) # index of 2d nodes based on number of nodes
nodes_2d = nodes[nidx_2d, :]     # coordinates of nodes


In [ ]:
# 1d network
ln = bmi.get_var('ln') 
ln_1d = np.array([[fr, to] for fr, to in ln if (fr in  cidx_1d) and (to in cidx_1d)])

In [ ]:
# 1d network
kn = bmi.get_var('kn') 
nlink_type = kn[:, 2] # type
nlinks = kn[:, :2] - 1 # link
nlinks_1d = nlinks[nlink_type == 1] # link_between_1D_nodes
nidx_1d = np.arange(nlinks_1d.min(), nlinks_1d.max()+1)
nodes_1d = nodes[nidx_1d, :]
nlinks_1d = nlinks_1d #- nlinks_1d.min()
# cidx_id  #
len(nidx_1d), len(cidx_1d)

### there is a difference between the 1d nodes and cells 
- number of 1d nodes (nk=9349) and 1d cells (nc=9425) is not the same, difference is boundary cells?
- how are the cells ordered. The first nk 1d cells are not the same as 1d nodes ??

In [ ]:
nidx_1d.size, cidx_1d.size # where does the difference come from -> downstream boundary nodes
# nidx_1d, cidx_1d

In [ ]:
cells_1d = cell_xy[cidx_1d[:nidx_1d.size]] # first 9349 cells (same number as 1d nodes)
np.all(nodes_1d == cells_1d)

In [ ]:
idx = np.where(np.not_equal(cells_1d, nodes_1d))[0]
idx

### link 1d network nodes to cell indices based on location

In [ ]:
# link 1d network nodes to cell indices based on location
ncidx_1d = np.ones_like(nidx_1d) * -1
import rtree
# create index
rt = rtree.index.Index()
for ci, xy in zip(cidx_1d, cell_xy[cidx_1d]):
    rt.insert(ci, xy.tolist())
# find nearest cell for each node
for ni, xy in enumerate(nodes[nidx_1d]):
    ci = list(rt.nearest(xy.tolist(), 1))
    assert np.all(xy == cell_xy[ci])
    ncidx_1d[ni] = ci[0] 
    # make sure each cell is found only ones
    # this is expensive, but only way to do it
    rt.delete(ci[0], xy.tolist()) 


In [ ]:
ncidx_1d = np.ma.masked_equal(ncidx_1d, -1)
ncidx_1d[~ncidx_1d.mask].size, np.unique(ncidx_1d[~ncidx_1d.mask]).size

In [ ]:
cells_1d = cell_xy[ncidx_1d] # index based on rtree
np.all(nodes_1d == cells_1d)

### find start and end points of 1d network based on link counts
- is ther another way to finde these?
- is up and downstream alway defined the same in links: from (downstream) -> to (upstream)


In [ ]:
# boundary conditions
bi = [ci for ci in cidx_1d if ci not in ncidx_1d]

In [ ]:
len(bi)

In [ ]:
# find start and end points of 1d network based on link counts
nidx_1d, nidx_1d_counts = np.unique(ln_1d, return_counts=True)
pidx_single = nidx_1d[nidx_1d_counts==1]
sp = np.array([p for p in pidx_single if p in ln_1d[0, :]])
ep = np.array([p for p in pidx_single if p not in sp])


In [ ]:
ln_1d_fix = np.array([[fr, to] for fr, to in ln_1d if (fr not in  sp) and (to not in ep)])

In [ ]:
len(ln_1d_fix), len(ln_1d)

In [ ]:
# reduce faces array to ragged array to be used as index
n_nodes_per_face = (~faces.mask).sum(axis=1)
ragged = [
    face[:n_nodes].filled()
    for n_nodes, face 
    in zip(n_nodes_per_face, faces)
]
fcoords = [nodes[np.append(face, face[0])] for face in ragged]
# faces

In [ ]:
# net links
# 0) closed_link_between_2D_nodes 
# 1) link_between_1D_nodes
# 2) link_between_2D_nodes
# 3) embedded_1D2D_link
# 4) 1D2D_link


# l1d2d = nlinks[nlink_type == 3] # link between 1D and 2D
# l1d2dcoords = nodes[l1d2d] #[nodes[link] for link in l1d]

## plots

In [ ]:
# create patches for faces of UGrid
fpatches = (matplotlib.patches.Polygon(face) for face in fcoords)
fpc = matplotlib.collections.PatchCollection(fpatches, edgecolor='grey')
fpc.set_facecolor('none')

In [ ]:
# line patches with links
# lcoords = nodes[nlinks_1d] #[nodes[link] for link in l1d]
lcoords = cell_xy[ln_1d_fix] 
lpc = matplotlib.collections.LineCollection(lcoords, color='green')
# l1d2d = matplotlib.collections.LineCollection(l1d2dcoords, color='m')

In [ ]:
# create basemap
%matplotlib notebook
plt.close()

# %%capture captured
fig, ax = make_map()
ax.add_collection(fpc)
ax.add_collection(lpc)
# ax.add_collection(l1d2d)

ax.autoscale()

In [ ]:
art_1dc, = ax.plot(cell_x[cidx_1d], cell_y[cidx_1d], '.m') # 1d cells
art_2dc, = ax.plot(cell_x[cidx_2d], cell_y[cidx_2d], '.b') # 2d cells
art_bi, = ax.plot(cell_x[cidx_bnds], cell_y[cidx_bnds], '.c') # boundary cells

In [ ]:
art_1dc.remove()
art_2dc.remove()
art_bi.remove()

In [ ]:
art_2dc, = ax.plot(cell_x[cidx_2d], cell_y[cidx_2d], '.b') # 1d nodes
# art_2d, = ax.plot(node_lon[nidx_2d], node_lat[nidx_2d], '.b') # 1d nodes

In [ ]:
art_2d.remove()
art_2dc.remove()

In [ ]:
art_ep, = ax.plot(node_lon[ep], node_lat[ep], '.r') # single end points
art_sp, = ax.plot(node_lon[sp], node_lat[sp], '.g') # single start points -> outlet

In [ ]:

art_1d.remove()
art_bi.remove()
art_ep.remove()
art_sp.remove()


## run and plot states

In [ ]:
# plot initial water level
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor('none')
wl = ax.scatter(cell_x[n2d:], cell_y[n2d:], c=cmap_wl(N(bmi.get_var('hs')[n2d:])), cmap=cmap_wl, s=3)
fig

In [ ]:
# corner admin links
# l1d = bmi.get_var('lncn')-1
# lcoords = nodes[l1d] #[nodes[link] for link in l1d]

In [ ]:
rain = bmi.get_var('rain')# np.zeros_like()
print(rain[ep].max())

In [ ]:
rain = np.zeros_like(bmi.get_var('rain'))
rain[ep] = 1000
bmi.set_var('rain', rain)

In [ ]:
for i in range(100):
    bmi.update()
bmi.get_current_time()/86400

In [ ]:
# bmi.get_var('q1')[ep]

In [ ]:
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor('none')
wl = ax.scatter(cell_x[n2d:], cell_y[n2d:], c=cmap_wl(N(bmi.get_var('hs')[n2d:])), cmap=cmap_wl, s=3)
fig

In [ ]:
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor(cmap_wl(N(bmi.get_var('hs')[:n2d])))
fig